# Metropolis Sampler Prototype

To Do:
* Prototype the multi-core functionality
* Improve likelihood implementation for $\tau$

### Initialize priors and parameter values

In [251]:
mu_a=0.7; 
mu_b=0.004;
theta_a=0.8; 
theta_b=10;
sigma_a=0.5; 
sigma_b=100;
Tlast = 30.;

In [252]:
theta = theta_a*theta_b; #0.105
sigma = sigma_a*sigma_b; #1.15
mu = mu_a*mu_b; #0.003

### Load likelihood codes and test data

In [253]:
include("/Users/nwelch/prelim/src/Julia/likelihood_functions.jl");

In [254]:
df = readtable("/Users/nwelch/prelim/data/infectionDataTest.csv");
dst = readtable("/Users/nwelch/prelim/data/infectedDistancesTest.csv");

In [255]:
head(df)

,x,y,t1,t2,t3,t4,t5,t6,tau,tauLowerBound,tauUpperBound
1,1.5,0.0,0,0,1,1,1,1,12.0,10,14
2,1.5,0.5,0,0,0,0,1,1,21.0,19,23
3,1.5,1.0,0,0,1,1,1,1,12.0,10,14
4,1.5,1.5,0,0,0,1,1,1,16.5,14,19
5,1.5,2.0,0,0,0,1,1,1,16.5,14,19
6,1.5,3.0,0,0,0,0,0,1,26.5,23,30


### Simulation Setup

In [256]:
trials = 100;
infectionCount = sum(df[:t6]);
chain = Matrix(trials, (3+infectionCount));
accept = fill(0, (3+infectionCount));

In [257]:
chain[1,1:infectionCount] = copy(df[:tau]);
chain[1,(infectionCount+1):(infectionCount+3)] = copy([mu sigma theta]);

In [258]:
U = Uniform();

In [259]:
for r=2:trials
    
    tau = copy(chain[r-1, 1:infectionCount])
    mu = copy(chain[(r-1), (infectionCount+1)])
    sigma = copy(chain[(r-1), (infectionCount+2)])
    theta = copy(chain[(r-1), (infectionCount+3)])
    
    
    # sample U to generate the acceptance criteria
    u = rand(U, (infectionCount+3))
    logU = log(u)
    
    
    # shared function arguments
    tauLessTIndex = find(x-> x<=Tlast, tau)
    tauGreaterTIndex = find(x-> x>Tlast, tau)
    
    
    # update mu
    mu_start = now()
    J_mu = Normal(mu, 0.0005)
    muStar = rand(J_mu, 1)[1]
    if muStar>0.0
        logLRatio_mu = llRatio_mu(mu, muStar, theta, sigma, tau, dst)
                              
        if logU[1] < logLRatio_mu
            mu = copy(muStar)
            accept[1] = accept[1] + 1
        end
    end
    mu_time = mu_time + (now() - mu_start);
  
    
    # update sigma
    sigma_start = now()
    J_sigma = Normal(sigma, 0.05)
    sigmaStar = rand(J_sigma, 1)[1]
    if sigmaStar>0.0
        
        gaussian_s = Normal(mu, sigma)
        gaussianStar_s = Normal(mu, sigmaStar)
        
        logLRatio_sigma = llRatio_sigma(sigma, sigmaStar,
            gaussian_s, gaussianStar_s, mu, tau, theta, dst)
        
        if logU[2] < logLRatio_sigma  
            sigma = copy(sigmaStar)
            accept[2] = copy(accept[2]) + 1
        end
    end
    sigma_time = sigma_time + (now() - sigma_start);
    
    
    # update theta
    theta_start = now()
    J_theta = Normal(theta, 0.005)
    thetaStar = rand(J_theta, 1)[1]
    if thetaStar>0.
        gaussian_t = Normal(mu, sigma)
        logLRatio_theta = llRatio_theta(theta, thetaStar,
            mu, tau, sigma, gaussian_t, dst)
        
        if logU[3] < logLRatio_theta 
            theta = copy(thetaStar)
            accept[3] = copy(accept[3]) + 1
        end 
    end
    theta_time = theta_time + (now() - theta_start);
    
    
    #tau update
    tau_start = now()
    gaussian_tau = Normal(mu, sigma)
    for i=1:infectionCount
        
        J_taui = Normal(tau[i], 1)
        tauiStar = rand(J_taui, 1)[1]

        lowerBound = df[:tauLowerBound][i]
        upperBound = df[:tauUpperBound][i]
        
        if (tauiStar<upperBound) & (tauiStar>lowerBound)
            logLRatio_tau = llRatio_tau(tau, tauiStar, i,
                mu, theta, sigma, gaussian_tau, dst)

            if logU[(i+3)] < logLRatio_tau
                tau[i] = copy(tauiStar)
                accept[i+3] = copy(accept[i+3]) + 1
            end
        end
    end
    tau_time = tau_time + (now() - tau_start);
    
    chain[r,1:infectionCount] = copy(tau);
    chain[r,(infectionCount+1):(infectionCount+3)] = copy([mu sigma theta]);
end

In [240]:
timing = [mu_time, sigma_time, tau_time]

3-element Array{Base.Dates.Millisecond,1}:
 101 milliseconds 
 183 milliseconds 
 3046 milliseconds

In [262]:
DataFrame(acceptCount = infectionCount)

,acceptCount
1,34


In [237]:
chainDF = DataFrame(chain);
chainColNames = [Symbol("tau$i") for i in 1:(infectionCount)];
chainColNames = append!(chainColNames, ["mu","sigma","theta"]);
names!(chainDF, chainColNames);

In [237]:
writetable("/Users/nwelch/prelim/data/mcmc_chain_julia.csv", 
    chainDF, separator = ',', header = true);